In [1]:
import os
import pandas as pd
import numpy as np

# sub_df = pd.read_csv('sub_data/all.csv')
vol_df = pd.read_csv('VolumeVoxel_df.csv')
# only CN and MCI spreadsheets have the image data id necessary to make the map
sub_df = pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - CN_All.csv').append(pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - MCI_All.csv')).append(pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - AD_All.csv'))

In [2]:
# extract just the problematic portion
# sub_df = sub_df.iloc[448:]
vol_df = vol_df.iloc[:423]

In [3]:
sub_df

,image data id,subject,class,sex,age,visit,modality,description,type,acq date,...,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,132220.0,002_S_1261,CN,F,73.0,4.0,MRI,MPR; ; N3; Scaled,Processed,5/27/2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,65561.0,002_S_1261,CN,F,71.0,2.0,MRI,MPR; ; N3; Scaled,Processed,2/27/2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,79135.0,002_S_1261,CN,F,72.0,3.0,MRI,MPR; ; N3; Scaled,Processed,8/30/2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,136188.0,002_S_1261,CN,F,73.0,6.0,MRI,MPR; ; N3; Scaled,Processed,2/05/2009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,78677.0,002_S_1280,CN,F,71.0,3.0,MRI,MPR; ; N3; Scaled,Processed,10/10/2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AD,F,72.0,4.0,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,11/27/2007,NiFTI,7/29/2019
76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AD,F,71.0,2.0,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,9/20/2006,NiFTI,7/29/2019
77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AD,F,72.0,4.0,MRI,MPR; ; N3; Scaled,Processed,12/07/2007,NiFTI,7/29/2019
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AD,F,71.0,2.0,MRI,MPR; ; N3; Scaled,Processed,12/18/2006,NiFTI,7/29/2019


In [4]:
# now I need to somehow build a map between rows... oh god
# loop through set of the truncated ids
truncIDs = set(sub_df['subject'])
# truncIDs

In [5]:
# all we care about in the vol df is the id
vol_ids = list(vol_df['filename'])

In [6]:
sub_df[sub_df.subject == '136_S_1227'].to_numpy()[:,5]

array([6.0, 5.0, 4.0, 2.0, 3.0], dtype=object)

In [7]:
godmap = {}
flagged = []
for truncID in truncIDs:
    print(truncID)
    sub_rows = sub_df[sub_df.subject == truncID]
    vids = [vid for vid in vol_ids if truncID in vid]
    if not len(sub_rows) == len(vids):
        print('WAT')
        continue
    # find coreespondence
    # first sort rows by visit
    sorted_subs = np.array(sorted(sub_rows.to_numpy(), key=lambda row: (row[5], row[0])))
    sorted_vids = sorted(vids, key=lambda vid: (vid[-3], vid[-1]))
    
    
    # if the second last digit is not one despite there being duplicate tweets... idk what to do.
    for i in range(len(vids)):
        godmap[vids[i]] = sorted_subs[i,0]
    if all([vid[-3] == '1' for vid in sorted_vids]):
        print('good.')
    else: # ...just sort by visit and hope it's fine?
        flagged = flagged + vids
            


    # break

nan


TypeError: 'in <string>' requires string as left operand, not float

In [ ]:
len(list(godmap.keys()))
# godmap.keys()
map_df = pd.DataFrame([[key, godmap[key], key in flagged] for key in godmap])
map_df.to_csv('id_map.csv')

In [ ]:
map_df

In [ ]:
raise Exception('stop')

In [ ]:
merged1 = pd.merge(sub_df, vol_df, how='left', on='filename')

In [ ]:
merged1

In [ ]:
# list(merged1['class'] == merged1['Target'])[:423]

In [ ]:
merged1.to_csv('merged1.csv')

In [ ]:
sub_id_col = vol_df['filename'].iloc[:423].apply(get_id)
vol_df['subject'] = sub_id_col

In [ ]:
vol_df

In [ ]:
sub_df

In [ ]:
merged2 = pd.merge(sub_df, vol_df, how='inner', on='subject')

In [ ]:
# list(merged2.columns)

In [ ]:
fff = merged2.drop_duplicates('filename_y')

In [ ]:
all(fff['class'] == fff['Target'])

In [ ]:
##########################

In [ ]:
worked = pd.merge(sub_df, vol_df_bottom, on='filename', how='left')

In [ ]:
worked[:448]

In [ ]:
# worked.to_csv('merged.csv')

In [ ]:
worked_top = worked.iloc[:448].copy(deep=True) #merge completed
worked_bottom = worked.iloc[448:].copy(deep=True)

In [ ]:
worked_bottom

In [ ]:
worked_bottom = pd.merge(worked_bottom, vol_df_top, how='outer', on='subject')

In [ ]:
worked_bottom

In [ ]:
worked = pd.merge(worked, vol_df_top, on='subject', how='left')

In [ ]:
worked.to_csv('merged.csv')